In [31]:
# If needed (uncomment to install):
# %pip install xlwings pandas numpy

import xlwings as xw
import pandas as pd
import numpy as np
from datetime import date, datetime



In [32]:
df = pd.read_excel('export.xls')
df

,Comapany,Account,Entry Date,Document Date,Document Type,Text,Document currency,Amount in doc. curr.,Local Currency,Amount in local currency,Year/month
0,UN0100,63010001.0,02.06.2020,01.06.2020,FZ,06015309EUB501 CHEURGE WEUVED EUSCENEU GLBEUL ...,USD,-1.005513e+05,USD,-100551.30,2020/06
1,XT0150,63010001.0,02.06.2020,01.06.2020,DZ,0601530BCC701 PRCEEDS F PURC,USD,8.446710e+03,USD,8446.71,2020/06
2,XT0150,63010001.0,02.06.2020,01.06.2020,FZ,0601530BCC701 PRCEEDS F PURCHEUSE F RECEVEUBLES,USD,-8.446710e+03,USD,-8446.71,2020/06
3,XT0150,63010001.0,02.06.2020,01.06.2020,DZ,0601530BDE801 PRCEEDS F PURC,USD,2.881539e+04,USD,28815.39,2020/06
4,XT0150,63010001.0,02.06.2020,01.06.2020,FZ,0601530BDE801 PRCEEDS F PURCHEUSE F RECEVEUBLES,USD,-2.881539e+04,USD,-28815.39,2020/06
...,...,...,...,...,...,...,...,...,...,...,...
1455,UN0100,63020002.0,30.06.2020,25.06.2020,TN,Petty cEUsh rembursement 25.06.2020,LKR,-1.113570e+05,USD,-598.02,2020/06
1456,UN0151,63020002.0,30.06.2020,25.06.2020,TN,Petty cEUsh rembursement 25.06.2020,LKR,-3.773300e+05,USD,-2026.37,2020/06
1457,UN0100,63010001.0,30.06.2020,30.06.2020,DZ,"SUMMERSEULT, NC TFR+ /UNFB/10874386-882//SUMME...",USD,1.454900e+04,USD,14549.00,2020/06
1458,UN0150,63010502.0,30.06.2020,29.06.2020,FZ,NaN,USD,3.486044e+04,USD,34860.44,2020/06


Helper functions (formatting, borders, autofit, gridlines)

In [33]:
# ------- Formatting helpers -------

LIGHT_BLUE = (221, 235, 247)  # a soft light blue for headers (RGB)

def remove_gridlines_for_active_window(app: xw.App):
    """Hide gridlines for the active window."""
    try:
        app.api.ActiveWindow.DisplayGridlines = False
    except Exception:
        pass

def set_header_style(header_rng: xw.Range):
    """Bold header row and set light-blue fill."""
    header_rng.api.Font.Bold = True
    # Color fill for header cells
    # Excel expects BGR when using .api? xlwings handles RGB via .color
    header_rng.color = LIGHT_BLUE



def apply_table_borders(rng: xw.Range):
    """Apply borders (no diagonal lines)."""
    # Excel border indices we want: 7=left, 8=top, 9=bottom, 10=right, 11=inside vertical, 12=inside horizontal
    for i in [7, 8, 9, 10, 11, 12]:
        b = rng.api.Borders(i)
        b.LineStyle = 1   # xlContinuous
        b.Weight = 2      # xlThin


def autofit_columns(ws: xw.Sheet, first_cell: str = "A1"):
    """Autofit all used columns on the sheet."""
    try:
        used = ws.used_range
        if used is not None:
            used.columns.autofit()
    except Exception:
        pass

def write_dataframe_as_table(ws: xw.Sheet, df: pd.DataFrame, top_left_cell: str = "B4"):
    start = ws.range(top_left_cell)

    # write headers + data
    ws.range(top_left_cell).value = [df.columns.tolist()] + df.values.tolist()

    nrows, ncols = df.shape
    header_rng = start.resize(1, ncols)          # <— exact header width
    set_header_style(header_rng)

    full_rng = start.resize(nrows + 1, ncols)    # <— exact table area (header + data)
    apply_table_borders(full_rng)
    return full_rng


def number_format_columns(ws, header_top_left, df, columns_to_format, num_format="#,##0.00"):
    start = ws.range(header_top_left)
    headers = list(df.columns)                   # <— no expand
    nrows = len(df)

    for col_name in columns_to_format:
        if col_name in headers:
            col_idx = headers.index(col_name)
            first_data_cell = start.offset(1, col_idx)
            last_data_cell  = first_data_cell.offset(nrows-1, 0)
            ws.range(f"{first_data_cell.address}:{last_data_cell.address}").number_format = num_format


def write_total_row(ws: xw.Sheet, header_top_left: str, df: pd.DataFrame,
                    label_col_name: str, sum_cols: list, bold=True):
    start   = ws.range(header_top_left)      # header top-left (e.g., B4)
    headers = list(df.columns)
    nrows   = len(df)
    ncols   = len(headers)

    # build a 1 x ncols range for the total row
    total_row = start.offset(nrows + 1, 0).resize(1, ncols)
    total_row.value = None  # clear the row

    # write "Total" label
    if label_col_name in headers:
        label_idx = headers.index(label_col_name)
        total_row[0, label_idx].value = "Total"

    # write SUM formulas in the requested numeric columns
    for col in sum_cols:
        if col in headers:
            idx = headers.index(col)
            col_top = start.offset(1, idx)            # first data cell in that column
            col_bot = col_top.offset(nrows - 1, 0)    # last data cell
            total_row[0, idx].formula = f"=SUM({col_top.address}:{col_bot.address})"

    if bold:
        total_row.api.Font.Bold = True

    # borders strictly around header + data + total (no extra columns)
    table_plus_total = start.resize(nrows + 2, ncols)
    apply_table_borders(table_plus_total)



def activate_and_hide_gridlines(app: xw.App, ws: xw.Sheet):
    ws.activate()
    remove_gridlines_for_active_window(app)


Build the workbook

In [34]:
# ---- Configuration: column selections ----
SUMMARY_COLS = [
    "Comapany",
    "Account",
    "Document currency",
    "Amount in doc. curr.",
    "Local Currency",
    "Amount in local currency",
]

ACCOUNT_COLS = [
    "Comapany",
    "Account",
    "Document Date",
    "Document Type",
    "Document currency",
    "Amount in doc. curr.",
    "Local Currency",
    "Amount in local currency",
    "Text",
]

SUM_COLS = ["Amount in doc. curr.", "Amount in local currency"]  # for totals in account sheets

# Ensure required columns exist (will raise if any missing)
missing_summary = [c for c in SUMMARY_COLS if c not in df.columns]
missing_account = [c for c in ACCOUNT_COLS if c not in df.columns]
if missing_summary or missing_account:
    raise ValueError(f"Missing columns. Summary missing: {missing_summary}; Account missing: {missing_account}")

In [35]:

# Coerce dates
if not np.issubdtype(df["Document Date"].dtype, np.datetime64):
    df["Document Date"] = pd.to_datetime(df["Document Date"], errors="coerce").dt.date

C:\Users\user\AppData\Local\Temp\ipykernel_29408\2572424626.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df["Document Date"] = pd.to_datetime(df["Document Date"], errors="coerce").dt.date


In [36]:
# Start Excel
app = xw.App(visible=True)
book = xw.Book()  # new unsaved workbook

# --- Create "Summary" first (reuse the first sheet to guarantee at least one visible sheet) ---
ws_sum = book.sheets[0]
ws_sum.name = "Summary"
ws_sum.clear()  # empty any default content

# Now it's safe to delete any other sheets (there will always be at least one: "Summary")
to_delete = [sh for sh in book.sheets if sh.name != "Summary"]
for sh in to_delete:
    sh.delete()

# -------- Summary content --------
today_str = date.today().strftime("%Y-%m-%d")
ws_sum["B2"].value = f"Document Ageing Report as at {today_str}"
ws_sum["B2"].api.Font.Bold = True
ws_sum["B2"].api.Font.Size = 14

summary_df = df[SUMMARY_COLS].copy()
tbl_rng = write_dataframe_as_table(ws_sum, summary_df, top_left_cell="B4")
number_format_columns(ws_sum, "B4", summary_df, columns_to_format=SUM_COLS, num_format="#,##0.00")
autofit_columns(ws_sum)
activate_and_hide_gridlines(app, ws_sum)

# ----------- ACCOUNT SHEETS -----------
unique_accounts = df["Account"].dropna().astype(str).unique()
for acc in sorted(unique_accounts):
    ws_acc = book.sheets.add(name=str(acc))  # new sheet per account

    acc_df = (
        df[df["Account"].astype(str) == str(acc)]
        .loc[:, ACCOUNT_COLS]
        .copy()
    )

    # --- Doc Ageing fix ---
    acc_df["Document Date"] = pd.to_datetime(acc_df["Document Date"], errors="coerce")
    acc_df["Doc Ageing"] = (pd.Timestamp(date.today()) - acc_df["Document Date"]).dt.days
    acc_df["Doc Ageing"] = acc_df["Doc Ageing"].fillna(0).astype("int64")

    # write table
    tbl_rng_acc = write_dataframe_as_table(ws_acc, acc_df, top_left_cell="B4")
    number_format_columns(ws_acc, "B4", acc_df, columns_to_format=SUM_COLS, num_format="#,##0.00")
    autofit_columns(ws_acc)

    # totals row
    write_total_row(
        ws_acc,
        header_top_left="B4",
        df=acc_df,
        label_col_name="Comapany",
        sum_cols=SUM_COLS,
        bold=True
    )

    activate_and_hide_gridlines(app, ws_acc)

ws_sum = book.sheets["Summary"]
ws_sum.api.Move(Before=book.sheets[0].api)  # puts Summary as first


# save
output_path = "outputfile/Final Report.xlsx"
book.save(output_path)
# Close the workbook & quit Excel (optional while testing)
book.close()
app.quit()

print(f"Workbook created and saved as: {output_path}")



Workbook created and saved as: outputfile/Final Report.xlsx
